
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Pinecone

In this section, we are going to try out another vector database, called Pinecone. It has a free tier which you need to sign up for to gain access below.

Pinecone is a cloud-based vector database that offers fast and scalable similarity search for high-dimensional data, with a focus on simplicity and ease of use. 

## Library pre-requisites

- pinecone-client
  - pip install below
- Spark connector jar file
  - **IMPORTANT!!** Since we will be interacting with Spark by writing a Spark dataframe out to Pinecone, we need a Spark Connector.
  - You need to attach a Spark-Pinecone connector `s3://pinecone-jars/spark-pinecone-uberjar.jar` in the cluster you are using. Refer to this [documentation](https://docs.pinecone.io/docs/databricks#setting-up-a-spark-cluster) if you need more information. 

In [0]:
%pip install pinecone-client==2.2.2


## Classroom Setup

In [0]:
%run ../Includes/Classroom-Setup



### Setting up your Pinecone

Step 1: Go to their [home page](https://www.pinecone.io/) and click `Sign Up Free` on the top right corner. 
<br>
Step 2: Click on `Sign Up`. It's possible that you may not be able to sign up for a new account, depending on Pinecone's availability. 

<img src="https://files.training.databricks.com/images/pinecone_register.png" width=300>

Step 3: Once you are in the console, navigate to `API Keys` and copy the `Environment` and `Value` (this is your API key).

<img src="https://files.training.databricks.com/images/pinecone_credentials.png" width=500>

In [0]:
# TODO
import os

os.environ["PINECONE_API_KEY"] = "<FILL IN>"
os.environ["PINECONE_ENV"] = "<FILL IN>"

In [0]:
import pinecone

pinecone_api_key = os.environ["PINECONE_API_KEY"]
pinecone_env = os.environ["PINECONE_ENV"]

pinecone.init(api_key=pinecone_api_key, environment=pinecone_env)

In [0]:
import pyspark.sql.functions as F

df = (
    spark.read.option("header", True)
    .option("sep", ";")
    .format("csv")
    .load(
        f"{DA.paths.datasets}/news/labelled_newscatcher_dataset.csv".replace(
            "/dbfs", "dbfs:"
        )
    )
    .withColumn("id", F.expr("uuid()"))
)
display(df)

For Pinecone, we need to generate the embeddings first and save it to a dataframe, before we can write it out to Pinecone for indexing. 

There are two ways of doing it: 
1. Using pandas DataFrame, apply the single-node embedding model, and upsert to Pinecone in batches
2. Using Spark Dataframe and use pandas UDFs to help us apply the embedding model on batches of data

### Method 1: Upsert to Pinecone in batches

In [0]:
pdf = df.limit(1000).toPandas()
display(pdf)

Note: Pinecone free tier only allows one index. If you have existing indices, you need to delete them before you are able create a new index.

We specify the similarity measure, embedding vector dimension within the index.

Read documentation on how to [create index here](https://docs.pinecone.io/reference/create_index/).

In [0]:
from sentence_transformers import SentenceTransformer

# We will use embeddings from this model to apply to our data
model = SentenceTransformer(
    "all-MiniLM-L6-v2", cache_folder=DA.paths.datasets
)  # Use a pre-cached model

Delete the index if it already exists

In [0]:
pinecone_index_name = "news"

if pinecone_index_name in pinecone.list_indexes():
    pinecone.delete_index(pinecone_index_name)

Create the index.

We specify the index name (required), embedding vector dimension (required), and a custom similarity metric (cosine is the default) when creating our index.

In [0]:
# only create index if it doesn't exist
if pinecone_index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=pinecone_index_name,
        dimension=model.get_sentence_embedding_dimension(),
        metric="cosine",
    )

# now connect to the index
pinecone_index = pinecone.Index(pinecone_index_name)

When the index has been created, we can now upsert vectors of data records to the index. `Upsert` means that we are writing the vectors into the index. 

Refer to this [documentation page](https://docs.pinecone.io/docs/python-client#indexupsert) to look at example code and vectors.

In [0]:
from tqdm.auto import tqdm

batch_size = 1000

for i in tqdm(range(0, len(pdf["title"]), batch_size)):
    # find end of batch
    i_end = min(i + batch_size, len(pdf["title"]))
    # create IDs batch
    ids = [str(x) for x in range(i, i_end)]
    # create metadata batch
    metadata = [{"title": title} for title in pdf["title"][i:i_end]]
    # create embeddings
    embedding_title_batch = model.encode(pdf["title"][i:i_end]).tolist()
    # create records list for upsert
    records = zip(ids, embedding_title_batch, metadata)
    # upsert to Pinecone
    pinecone_index.upsert(vectors=records)

# check number of records in the index
pinecone_index.describe_index_stats()

Once the vectors are upserted, we can now query the index directly! Notice that it returns us the similarity score in the result too.

In [0]:
query = "fish"

# create the query vector
user_query = model.encode(query).tolist()

# submit the query to the Pinecone index
pinecone_answer = pinecone_index.query(user_query, top_k=3, include_metadata=True)

for result in pinecone_answer["matches"]:
    print(f"{round(result['score'], 2)}, {result['metadata']['title']}")
    print("-" * 120)

### Method 2: Process with Spark and write to Pinecone with Spark

Now that we have seen how to `upsert` with Pinecone, you may be curious whether we can use Spark DataFrame Writer (just like Weaviate) to write the entire dataframe out in a single operation. The answer is yes -- we will now take a look at how to do that and a spoiler alert is that you will need to use a Spark connector too! 

We first need to write a mapping function to map the tokenizer and embedding model onto batches of rows within the Spark DataFrame. We will be using a type of [pandas UDFs](https://www.databricks.com/blog/2020/05/20/new-pandas-udfs-and-python-type-hints-in-the-upcoming-release-of-apache-spark-3-0.html), called scalar iterator UDFs. 

> The function takes and outputs an iterator of pandas.Series.

> The length of the whole output must be the same length of the whole input. Therefore, it can prefetch the data from the input iterator as long as the lengths of entire input and output are the same. The given function should take a single column as input.

> It is also useful when the UDF execution requires expensive initialization of some state. 

We load the model once per partition of data, not per [batch](https://spark.apache.org/docs/latest/api/python/user_guide/sql/arrow_pandas.html#setting-arrow-batch-size), which is faster. 

For more documentation, refer [here](https://docs.databricks.com/udf/pandas.html).

In [0]:
import pandas as pd
from pyspark.sql.functions import pandas_udf
from sentence_transformers import SentenceTransformer
from typing import Iterator

@pandas_udf("array<float>")
def create_embeddings_with_transformers(
    sentences: Iterator[pd.Series],) -> Iterator[pd.Series]:
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    for batch in sentences:
        yield pd.Series(model.encode(batch).tolist())

In [0]:
import pyspark.sql.functions as F

transformer_type = "sentence-transformers/all-MiniLM-L6-v2"
embedding_spark_df = (
    df.limit(1000)
    .withColumn("values", create_embeddings_with_transformers("title")) 
    .withColumn("namespace", F.lit(transformer_type))
    .withColumn("metadata", F.to_json(F.struct(F.col("topic").alias("TOPIC"))))
    # We select these columns because they are expected by the Spark-Pinecone connector
    .select("id", "values", "namespace", "metadata")
)
display(embedding_spark_df)

Repeat the same step as in Method 1 above to delete and recreate the index. Again, we need to delete the index because Pinecone free tier only allows one index.

Note: This could take ~3 minutes. 

In [0]:
pinecone_index_name = "news"

if pinecone_index_name in pinecone.list_indexes():
    pinecone.delete_index(pinecone_index_name)

# only create index if it doesn't exist
model = SentenceTransformer(transformer_type)
if pinecone_index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=pinecone_index_name,
        dimension=model.get_sentence_embedding_dimension(),
        metric="cosine",
    )

# now connect to the index
pinecone_index = pinecone.Index(pinecone_index_name)

Instead of writing in batches, you can now use Spark DataFrame Writer to write the data out to Pinecone directly.

**IMPORTANT!!** You need to attach a Spark-Pinecone connector `s3://pinecone-jars/spark-pinecone-uberjar.jar` in the cluster you are using. Otherwise, this following command would fail. Refer to this [documentation](https://docs.pinecone.io/docs/databricks#setting-up-a-spark-cluster) if you need more information. 

In [0]:
(
    embedding_spark_df.write.option("pinecone.apiKey", pinecone_api_key)
    .option("pinecone.environment", pinecone_env)
    .option("pinecone.projectName", pinecone.whoami().projectname)
    .option("pinecone.indexName", pinecone_index_name)
    .format("io.pinecone.spark.pinecone.Pinecone")
    .mode("append")
    .save()
)

&copy; 2023 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>